In [1]:
import pandas as pd
import numpy as np


from sklearn.metrics import make_scorer, f1_score, precision_score, recall_score, accuracy_score, roc_curve, roc_auc_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier

from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from scipy.stats import uniform
import matplotlib.pyplot as plt

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

import xgboost as xgb

In [2]:
raw_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/Train_raw.csv')
raw_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/Test_raw.csv')

Adasyn_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/Adasyn_train.csv')
Adasyn_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/Adasyn_test.csv')

Nearmiss_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/Nearmiss_train.csv')
Nearmiss_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/Nearmiss_test.csv')

ROS_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/ROS_train.csv')
ROS_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/ROS_test.csv')

RUS_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/RUS_train.csv')
RUS_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/RUS_test.csv')

SMOTETomek_train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/SMOTETomek_train.csv')
SMOTETomek_test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Cancellation/data/datasets/SMOTETomek_test.csv')

In [3]:
pip install neptune-client

In [4]:
import neptune

API_TOKEN='' #@param {type:"string"}

PROJEKT_NAME = 'DataWorkshop-Foundation/Predicting-cancellation-of-visit' 

neptune.init(
   api_token=API_TOKEN,
   project_qualified_name=PROJEKT_NAME
)

Project(DataWorkshop-Foundation/Predicting-cancellation-of-visit)

In [5]:
raw_train.head()

,PatientId,Gender,Age,Neighbourhood_0,Neighbourhood_1,Neighbourhood_2,Neighbourhood_3,Neighbourhood_4,Neighbourhood_5,Neighbourhood_6,Neighbourhood_7,Scholarship,Hypertension,Diabetes,Alcoholism,Handicap,SMSReceived,MonthBookingDate_0,MonthBookingDate_1,MonthBookingDate_2,MonthBookingDate_3,DayBookingDate_0,DayBookingDate_1,DayBookingDate_2,DayBookingDate_3,DayBookingDate_4,DayBookingDate_5,MonthAppointmentDate_0,MonthAppointmentDate_1,MonthAppointmentDate_2,DayAppointmentDate_0,DayAppointmentDate_1,DayAppointmentDate_2,DayAppointmentDate_3,DayAppointmentDate_4,DayAppointmentDate_5,HourOfDayBooking_0,DaysToAppointment,AppointmentWeekDay_0,AppointmentWeekDay_1,AppointmentWeekDay_2,AppointmentWeekDay_3,NoShow
0,4652519881919,0,8,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,1,0,1,0,1,0,0,0,0,0,1,0,1,1,0,0,1,0,0
1,81431485833737,0,74,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,12,0,0,1,1,0
2,555956674526286,0,50,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,31,0,0,0,1,0
3,14559326134688,0,21,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,5,0,0,1,0,0
4,16796474433112,0,39,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,1,0,1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,4,0,0,0,1,1


In [6]:
!pip install eli5
!pip install hyperopt

     |████████████████████████████████| 112kB 12.3MB/s 


In [7]:
def get_feats(df):
    feats = df.select_dtypes(include=[np.int, np.float]).columns 
    return feats[ (feats != 'NoShow')].values

def get_X(df):
    return df[ get_feats(df) ].values

def get_y(df, target_var='NoShow'):
    return df[target_var].values

In [8]:
feats = ['PatientId',
 	'Gender',
  'Age', 	
  'Neighbourhood_0', 	
  'Neighbourhood_1', 	
  'Neighbourhood_2', 	
  'Neighbourhood_3',	
  'Neighbourhood_4', 	
  'Neighbourhood_5', 
  'Neighbourhood_6', 
  'Neighbourhood_7',	
  'Scholarship', 	
  'Hypertension', 	
  'Diabetes', 	
  'Alcoholism', 	
  'Handicap', 	
  'SMSReceived', 	
  'MonthBookingDate_0', 	
  'MonthBookingDate_1',	
  'MonthBookingDate_2', 	
  'MonthBookingDate_3', 	
  'DayBookingDate_0', 	
  'DayBookingDate_1',	
  'DayBookingDate_2',	
  'DayBookingDate_3', 	
  'DayBookingDate_4', 	
  'DayBookingDate_5', 	
  'MonthAppointmentDate_0', 	
  'MonthAppointmentDate_1', 	
  'MonthAppointmentDate_2', 	
  'DayAppointmentDate_0',	
  'DayAppointmentDate_1',	
  'DayAppointmentDate_2',	
  'DayAppointmentDate_3', 	
  'DayAppointmentDate_4',
  'DayAppointmentDate_5', 	
  'HourOfDayBooking_0', 	
  'DaysToAppointment', 	
  'AppointmentWeekDay_0', 	
  'AppointmentWeekDay_1', 	
  'AppointmentWeekDay_2', 	
  'AppointmentWeekDay_3']

In [9]:
X_raw = get_X(raw_train)
y_raw = get_y(raw_train)

In [10]:
len(feats)

42

In [11]:
xgb_reg_params = {
    'learning_rate':    hp.choice('learning_rate',    np.arange(0.05, 0.31, 0.05)),
    'max_depth':        hp.choice('max_depth',        np.arange(5, 16, 1, dtype=int)),
    'min_child_weight': hp.choice('min_child_weight', np.arange(1, 8, 1, dtype=int)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3, 0.8, 0.1)),
    'subsample':        hp.uniform('subsample', 0.8, 1),
    'n_estimators':     50,
}

In [12]:
def run_model(model):

  scores = cross_val_score(model, X_raw, y_raw, cv = 3, scoring='f1')
  
  return np.mean(scores), np.std(scores)

def hyperopt_train_test(params):
 

  print("Training with params: ")
  print(params)
  clf = xgb.XGBClassifier(**params)
  mean_f1, score_std = run_model(clf)
  print('loss: ',np.abs(mean_f1))
  return np.abs(mean_f1)

count = 0
best = 0

def f(params):
  f1 = hyperopt_train_test(params)
  return {'loss': -f1, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, xgb_reg_params, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=50, trials=trials)
print('best:')
print(best)

NameError: ignored

In [ ]:
best

In [ ]:
parameters = ['max_depth', 'colsample_bytree', 'learning_rate', 'min_child_weight', 'subsample'] # decision tree
cols = len(parameters)

f, axes = plt.subplots(nrows=1, ncols=cols, figsize=(20,5))

cmap = plt.cm.jet 

for i, val in enumerate(parameters):
    xs = np.array([t['misc']['vals'][val] for t in trials.trials]).ravel()
    ys = [-t['result']['loss'] for t in trials.trials]
   # xs, ys = zip(\*sorted(zip(xs, ys))
    ys = np.array(ys)
    axes[i].scatter(xs, ys, s=20, linewidth=0.01, alpha=0.5, c=cmap(float(i)/len(parameters)))
    axes[i].set_title(val)
    #axes[i].set_ylim([0.9,1.0])

In [ ]:
clf =  xgb.XGBClassifier(**best)

In [ ]:
clf

In [ ]:
def model(train, test, search, name, tags):
  
  #print(search.get_params())
  y_train = train["NoShow"]
  y_test = test["NoShow"]

  X_train = train.loc[:, train.columns != 'NoShow']
  X_test = test.loc[:, test.columns != 'NoShow']
                
  search = search.fit(X_train, y_train)

  #search.best_params_

  y_pred = search.predict(X_test)
  print(y_pred)
  accuracy = accuracy_score(y_test, y_pred)
  print("Accuracy: %.2f%%" % (accuracy * 100.0))
  print(' f1 ', f1_score(y_test, y_pred), '\n',
        'recall', recall_score(y_test, y_pred), '\n',
        'precision', precision_score(y_test, y_pred) )

  neptune.create_experiment(
      name=name
  )

  neptune.log_metric('accuracy', accuracy)
  neptune.log_metric('f1', f1_score(y_test, y_pred))
  neptune.log_metric('recall', recall_score(y_test, y_pred))
  neptune.log_metric('precision', precision_score(y_test, y_pred))
  neptune.append_tags(tags)
  neptune.append_tags([str(best),"Magdalena Cebula"])
  neptune.append_tags([str(feats)])
  #neptune.append_tags([str(space4dt)])
  from sklearn.metrics import confusion_matrix

  conf_mat = confusion_matrix(y_true=y_test, y_pred=y_pred)
  print('Confusion matrix:\n', conf_mat)

  from matplotlib import pyplot as plt

  labels = ['Class 0', 'Class 1']
  fig = plt.figure()
  ax = fig.add_subplot()
  cax = ax.matshow(conf_mat, cmap=plt.cm.Blues)
  fig.colorbar(cax)
  ax.set_xticklabels([''] + labels)
  ax.set_yticklabels([''] + labels)
  plt.xlabel('Predicted')
  plt.ylabel('Expected')
  plt.show()

  neptune.log_image('Confusion_matrix ' + name, fig)
  
  false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, search.predict(X_test))
  print(false_positive_rate, true_positive_rate, thresholds)


  roc_auc_score(y_test, y_pred)
  auc = roc_auc_score(y_test, y_pred)
  neptune.log_metric('roc_auc', auc)
  fig = plt.figure()
  plt.plot(false_positive_rate,true_positive_rate,label="data 1, auc="+str(auc))
  plt.legend(loc=4)
  plt.show()
  neptune.log_image('ROC AUC ' + name, fig)
  neptune.stop()

In [ ]:
model(raw_train, raw_test, clf, 'XGBClassifier', ['XGBClassifier', 'Raw'])

In [ ]:
X_neamiss = get_X(Nearmiss_train)
y_neamiss = get_y(Nearmiss_train)

def run_model(model):

  scores = cross_val_score(model, X_neamiss, y_neamiss, cv = 3, scoring='f1')
  
  return np.mean(scores), np.std(scores)

In [ ]:
def hyperopt_train_test(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBClassifier(**params))
  print('loss: ',np.abs(mean_mae))
  return np.abs(mean_mae)

def f(params):
  f1 = hyperopt_train_test(params)
  return {'loss': -f1, 'status': STATUS_OK}
  
trials = Trials()

best = fmin(f, xgb_reg_params, algo=tpe.suggest, max_evals=50, trials=trials)

In [ ]:
print('best:')
print(best)

In [ ]:
clf = criterion(best)

In [ ]:
model(Nearmiss_train, Nearmiss_test,clf, 'XGBClassifier', ['XGBClassifier', 'Nearmiss'])

In [ ]:
X_ros = get_X(ROS_train)
y_ros = get_y(ROS_train)

def run_model(model):

  scores = cross_val_score(model, X_ros, y_ros, cv = 3, scoring='f1')
  
  return np.mean(scores), np.std(scores)

def hyperopt_train_test(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBClassifier(**params))
  print('loss: ',np.abs(mean_mae))
  return np.abs(mean_mae)

def f(params):
  f1 = hyperopt_train_test(params)
  return {'loss': -f1, 'status': STATUS_OK}
  
trials = Trials()

best = fmin(f, xgb_reg_params, algo=tpe.suggest, max_evals=50, trials=trials)

In [ ]:
print(best)

In [ ]:
clf = criterion(best)

In [ ]:
model(ROS_train, ROS_test,clf, 'XGBClassifier', ['XGBClassifier', 'ROS'])

In [ ]:
X_rus = get_X(RUS_train)
y_rus = get_y(RUS_train)

def run_model(model):

  scores = cross_val_score(model, X_rus, y_rus, cv = 3, scoring='f1')
  
  return np.mean(scores), np.std(scores)

def hyperopt_train_test(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBClassifier(**params))
  print('loss: ',np.abs(mean_mae))
  return np.abs(mean_mae)

def f(params):
  f1 = hyperopt_train_test(params)
  return {'loss': -f1, 'status': STATUS_OK}
  
trials = Trials()

best = fmin(f, xgb_reg_params, algo=tpe.suggest, max_evals=50, trials=trials)

In [ ]:
print(best)

In [ ]:
clf = criterion(best)

In [ ]:
model(RUS_train, RUS_test,clf, 'XGBClassifier', ['XGBClassifier', 'RUS'])

In [ ]:
X_tomek = get_X(SMOTETomek_train)
y_tomek = get_y(SMOTETomek_train)

def run_model(model):

  scores = cross_val_score(model, X_tomek, y_tomek, cv = 3, scoring='f1')
  
  return np.mean(scores), np.std(scores)

def hyperopt_train_test(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBClassifier(**params))
  print('loss: ',np.abs(mean_mae))
  return np.abs(mean_mae)

def f(params):
  f1 = hyperopt_train_test(params)
  return {'loss': -f1, 'status': STATUS_OK}
  
trials = Trials()
best = None
best = fmin(f, xgb_reg_params, algo=tpe.suggest, max_evals=50, trials=trials)

In [ ]:
print(best)

In [ ]:
clf = xgb.XGBClassifier(best)

In [ ]:
model(SMOTETomek_train, SMOTETomek_test,clf, 'XGBClassifier', ['XGBClassifier', 'SMOTETomek'])

In [ ]:
X_adasyn  = get_X(Adasyn_train)
y_adasyn = get_y(Adasyn_train)

def run_model(model):

  scores = cross_val_score(model, X_adasyn, y_adasyn, cv = 3, scoring='f1')
  
  return np.mean(scores), np.std(scores)

def hyperopt_train_test(params):
  print("Training with params: ")
  print(params)

  mean_mae, score_std = run_model(xgb.XGBClassifier(**params))
  print('loss: ',np.abs(mean_mae))
  return np.abs(mean_mae)

def f(params):
  f1 = hyperopt_train_test(params)
  return {'loss': -f1, 'status': STATUS_OK}
  
trials = Trials()

best = fmin(f, xgb_reg_params, algo=tpe.suggest, max_evals=50, trials=trials)

In [ ]:
clf = criterion(best2)

In [ ]:
model(Adasyn_train, Adasyn_test,clf,'XGBClassifier', ['XGBClassifier', 'Adasyn'])